# KGOntologies Example N#1
## AcmeBank Stafflist

Load a data file containing members of staff and the departments in which they are employed and process it into a marked-up knowledge-graph for visualisation. 



In [1]:
import os, sys

srcdir=r'..'
if srcdir not in sys.path:
    sys.path.append(srcdir)

from IPython.display import display, Image, HTML
import pandas as pd
import src.graphvisutils_gravis as graphvisutils_gravis 
import src.graphloader as graphloader
import src.queryaugment as queryaugment
import gravis as gv

import rdflib
from rdflib import URIRef, Literal, Namespace
import uuid
from hashlib import md5
import json
from functools import partial

/home/tomk/.local/share/virtualenvs/kgontologies-24Yr4f6_/lib/python3.10/site-packages/gravis/_internal/plotting/template_system.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as _pkg_resources


In [2]:
# Locate and read the data file
data_df = pd.read_excel("../data/AcmeBankStaffList.xlsx", na_values='None')

In [3]:
# Load and bind associated ontologies for additional data enrichment
kgdmcar_g = rdflib.Graph()
kgdmcar_g.parse ('../ontologies/kgdmcar.owl', format='xml')

kgnaming_g = rdflib.Graph()
kgnaming_g.parse ('../ontologies/kgnaming.owl', format='xml')

ontology_graph = rdflib.Graph(bind_namespaces="rdflib")
for t in kgnaming_g.triples((None, None, None)):
    ontology_graph.add(t)

for t in kgdmcar_g.triples((None, None, None)):
    ontology_graph.add(t)

In [38]:
# Convert the data into a raw graph format
rawg=graphloader.rdflib_graph_from_dataframe(data_df[0:500])

# Open a serialisation config file and convert the raw graph into triples describing entities, relations and literals
serial_config = json.load(open("../serialisations/acmestaff_serialisation.json", "r"))
ents,rels,lits  = graphloader.process_anonymous_data_graph(rawg, serial_config, data_namespace="http://data#")

# Aggregate the triples into a single list
triple_collection = [t for e in ents for t in e.to_triples() ]
triple_collection.extend([t for r in rels for t in r.to_triples()])
triple_collection.extend([t for l in lits for t in l.to_triples()])

# Define a target graph and bind associated namespaces to it for url-shortening
load_graph = rdflib.Graph(bind_namespaces="rdflib")
KGNAM = Namespace("http://www.semanticweb.org/tomk/ontologies/2025/5/kgnaming#")
DMCAR = Namespace("http://www.semanticweb.org/tomk/ontologies/2025/5/kgdmcar#")
ACME = Namespace("http://www.acmebank.org/")
load_graph.bind("kgnam", KGNAM)
load_graph.bind("dmcar", DMCAR)
load_graph.bind("acme", ACME)

# Load the triples into the graph
for t in triple_collection:
    load_graph.add(t)

Person : <root>
Department : <root>
Role : Department
##############################
ACME0000 : ACME0000, 
ACME0001 : ACME0001, 
ACME0002 : ACME0002, 
ACME0003 : ACME0003, 
ACME0004 : ACME0004, 
ACME0005 : ACME0005, 
ACME0006 : ACME0006, 
ACME0007 : ACME0007, 
ACME0008 : ACME0008, 
ACME0009 : ACME0009, 
ACME0010 : ACME0010, 
ACME0011 : ACME0011, 
ACME0012 : ACME0012, 
ACME0013 : ACME0013, 
ACME0014 : ACME0014, 
ACME0015 : ACME0015, 
ACME0016 : ACME0016, 
ACME0017 : ACME0017, 
ACME0018 : ACME0018, 
ACME0019 : ACME0019, 
ACME0020 : ACME0020, 
ACME0021 : ACME0021, 
ACME0022 : ACME0022, 
ACME0023 : ACME0023, 
ACME0024 : ACME0024, 
ACME0025 : ACME0025, 
ACME0026 : ACME0026, 
ACME0027 : ACME0027, 
ACME0028 : ACME0028, 
ACME0029 : ACME0029, 
ACME0030 : ACME0030, 
ACME0031 : ACME0031, 
ACME0032 : ACME0032, 
ACME0033 : ACME0033, 
ACME0034 : ACME0034, 
ACME0035 : ACME0035, 
ACME0036 : ACME0036, 
ACME0037 : ACME0037, 
ACME0038 : ACME0038, 
ACME0039 : ACME0039, 
ACME0040 : ACME0040, 
ACME0041 : AC

In [39]:
# Set default visualiser settings
# Layout algorithm options:
#    “barnesHut”, “forceAtlas2Based”, “repulsion”, “hierarchicalRepulsion”
vis_graph_settings = {
    "node_label_data_source" : "label", 
    "node_size_data_source" : "size", 
    "node_label_size_factor" : 1.5, 
    "node_size_factor" : 2.5, 
    "edge_size_data_source" : "size", 
    "edge_label_data_source" : "label", 
    "edge_size_factor" : 2.5, 
    "show_edge_label" : True,
    "edge_curvature" : 0.2,
    "graph_height" : 800,
    "details_height" : 300, 
    "layout_algorithm" : "forceAtlas2Based", 
    #"layout_algorithm" : "hierarchicalRepulsion", 
    #"gravitational_constant" : -0.15, 
    "central_gravity" : 2.0, 
    "spring_constant" : 0.14,
    "avoid_overlap" : 0.5
    
}

d3_graph_settings = {
    "node_label_data_source" : "label", 
    "node_size_data_source" : "size", 
    "edge_size_data_source" : "size", 
    "edge_label_data_source" : "label", 
    "show_edge_label" : True,
    "edge_curvature" : 0.2,
    "use_collision_force" : True, 
    "collision_force_radius" : 60, 
    "collision_force_strength" : 0.8,
    "graph_height" : 800,
    "details_height" : 300, 
    
}

In [40]:
# Generate a networkx "property-graph" representation of the knowledge=graph for passing to the visualiser
load_nx_g = graphvisutils_gravis.rdflib_graph_to_networkx_for_gravis(load_graph, 
                                                                ontology_graph, 
                                                                hide_types=True, 
                                                                hide_literals=True, 
                                                                node_colour_scheme="bold4")

In [41]:
gv.vis(load_nx_g, **vis_graph_settings)

In [42]:
# Run a SPARQL Query to Retrieve only Active Staff Members (i.e. those with an assigned role)
# Note that this query is only returning nodes that meed certain conditions, the resulting
# resultset is not in s,p,o form, just contains a collection of node-uris.

active_staff_q = """
PREFIX acme: <http://www.acmebank.org/>

SELECT ?s ?q
WHERE 
    {
    
    {SELECT ?s ?q
    WHERE { 
    ?s a ?type.
    ?s acme:HasRole ?q.
    ?q a acme:Role.
    VALUES ?type {acme:Person} .
    }}

UNION

    {SELECT ?s
    WHERE { 
    ?s a ?type.
    VALUES ?type {acme:Role} .
    }}

UNION
    
    {SELECT ?s
    WHERE { 
    ?s a ?type.
    VALUES ?type {acme:Department} .
    }}

}


"""


In [43]:
# Run the query, then augment it with the get_x options, this allows for simpler workflow and easier sparql queries to be performed
# Augmentation results in a new subgraph, which is driven from the parameters
subgraph=queryaugment.augment_sparql_to_graph(load_graph, 
                          active_staff_q, 
                          get_types=True, 
                          get_labels=True, 
                          get_literals=True, 
                          get_cross_links=True, 
                          sparql_option=False)

In [44]:
# Bind some appropriate namespaces to the resulting subgraph object
KGNAM = Namespace("http://www.semanticweb.org/tomk/ontologies/2025/5/kgnaming#")
DMCAR = Namespace("http://www.semanticweb.org/tomk/ontologies/2025/5/kgdmcar#")
ACME = Namespace("http://www.acmebank.org/")
subgraph.bind("kgnam", KGNAM)
subgraph.bind("dmcar", DMCAR)
subgraph.bind("acme", ACME)

In [45]:
# Generate a networkx "property-graph" representation of the knowledge=graph for passing to the visualiser
nx_g = graphvisutils_gravis.rdflib_graph_to_networkx_for_gravis(subgraph, 
                                                                ontology_graph, 
                                                                hide_types=True, 
                                                                hide_literals=True, 
                                                                node_colour_scheme="bold4")

In [46]:
# Set default visualiser settings
# Layout algorithm options:
#    “barnesHut”, “forceAtlas2Based”, “repulsion”, “hierarchicalRepulsion”
vis_graph_settings = {
    "node_label_data_source" : "label", 
    "node_size_data_source" : "size", 
    "node_label_size_factor" : 1.5, 
    "node_size_factor" : 2.5, 
    "edge_size_data_source" : "size", 
    "edge_label_data_source" : "label", 
    "edge_size_factor" : 2.5, 
    "show_edge_label" : True,
    "edge_curvature" : 0.2,
    "graph_height" : 800,
    "details_height" : 300, 
    "layout_algorithm" : "forceAtlas2Based", 
    #"layout_algorithm" : "hierarchicalRepulsion", 
    #"gravitational_constant" : -0.15, 
    "central_gravity" : 2.0, 
    "spring_constant" : 0.14,
    "avoid_overlap" : 0.5
    
}

d3_graph_settings = {
    "node_label_data_source" : "label", 
    "node_size_data_source" : "size", 
    "edge_size_data_source" : "size", 
    "edge_label_data_source" : "label", 
    "show_edge_label" : True,
    "edge_curvature" : 0.2,
    "use_collision_force" : True, 
    "collision_force_radius" : 60, 
    "collision_force_strength" : 0.8,
    "graph_height" : 800,
    "details_height" : 300, 
    
}

In [47]:
gv.vis(nx_g, **vis_graph_settings)